In [1]:
import pypylon.pylon as py
import matplotlib.pyplot as plt
import numpy as np


In [79]:
%%timeit

def Initcam():
    tlf = py.TlFactory.GetInstance()
    devices = tlf.EnumerateDevices()
    
    mycam = py.InstantCamera(tlf.CreateDevice(devices[0]))
    
    return (mycam)

25 ns ± 0.0564 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [80]:
def acquire(mycam):
    result = mycam.GrabOne(100)
    myimg = result.GetArray()
    return(myimg)


In [81]:
camera1 = Initcam() #this takes about 300ms...
camera1.Open()      #this takes about 200 ms

In [82]:

image = acquire(camera1) #this takes about 300 ms. 

In [83]:
print(type(image))

<class 'numpy.ndarray'>


In [84]:
np.shape(image)

(4600, 5280)

In [85]:
from scipy import ndimage
def get_centroid (ndarray):
    centroid = ndimage.center_of_mass((ndarray))
    return (centroid)

In [86]:

center = get_centroid(image) #this takes about 110 ms.

In [87]:
center

(2281.443280792328, 2630.305159147291)

In [50]:
test = np.shape(image)
print(test)

(4600, 5280)


In [69]:
alan = np.zeros((test))

In [70]:
print(alan)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [64]:
print(np.shape(alan/2))

(4600, 5280)


In [78]:
%%timeit

alan = np.zeros((4600,5280))
for i in range (0,10):
    alan = alan + acquire(camera1)
    


3.56 s ± 19.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
